In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import scipy

import numpy as np
import pandas as pd
import xarray as xr
import collections

from xsd.pointwise_models import BcsdTemperature, BcsdPrecipitation

In [2]:
# open a small dataset for training
training = xr.open_zarr('/home/jovyan/xsd/data/downscale_test_data.zarr.zip', group='training')

# open a small dataset of observations (targets)
targets = xr.open_zarr('/home/jovyan/xsd/data/downscale_test_data.zarr.zip', group='targets')

In [3]:
X_daily = training.isel(point=0).to_dataframe()[['T2max']]
# convert y in K as well 
y_daily = targets.isel(point=0).to_dataframe()[['Tmax']] + 273.15

In [4]:
MONTH_GROUPER = lambda x: x.month

In [5]:
isinstance(MONTH_GROUPER, str)

False

In [6]:
type(MONTH_GROUPER)

function

In [7]:
callable(MONTH_GROUPER)

True

In [8]:
# initialize the BCSD Temperature model
time_grouper = 'daily_nasa-nex'
bcsd_temp = BcsdTemperature(time_grouper=time_grouper)

In [9]:
bcsd_temp

BcsdTemperature(time_grouper=<class 'xsd.pointwise_models.groupers.PaddedDOYGrouper'>)

In [10]:
bcsd_temp.fit(X_daily, y_daily)

BcsdTemperature(time_grouper=<class 'xsd.pointwise_models.groupers.PaddedDOYGrouper'>)

In [ ]:
bcsd_temp.fit(X_temp, y_temp)
out = bcsd_temp.predict(X_temp) + X_temp
plot_cdf(X=X_temp, y=y_temp, out=out)
out.plot()